# Facebook Marketing Project: Data Analytics Report

* Project Source: Some Company <br>
* Creator: Cheng Chen <br>
* Date: 07/07/2018

### Outline of Analytics Report
1. Data Cleaning & Exploration <br>
    1.1 Configurate the environment & load datasets <br>
    1.2 Define some functions <br>
    1.3 Clean dataset - dat_repay <br>
    1.4 Clean dataset - dat_ppl <br>
    1.5 Clean dataset - dat_dev <br>
    1.6 Clean dataset - dat_mpesa <br>
 <br>    
2. Modeling the Likelihood of Repayment <br>
    2.1 Prepare the combined dataset <br>
    2.2 Modeling the likelihood of repayment <br>
    2.3 Support the decisions <br>
    2.4 Understand the customers through dat_mpesa <br>
 <br>    
3. Modeling the Life-Time-Value (LTV) of Borrowers <br>
    3.1 Prepare the dataset: dat_ltv <br>
    3.2 Modeling the LTV using logistic regression <br>
    3.3 Support the decisions <br>
    3.4 Understand the customers through dat_mpesa <br>
 <br>   
4. Summary of Recommendations <br>
    4.1 Recommendation - User Acquisition <br>
    4.2 Recommendation - Loan Application and Product <br>

### Basic Value Propositions and Assumptions
1. If a customer repays his/her first loan, it means he/she either is a good customer or has the need to make loans again in the future. We will analyze the likelihood of repayment based on repayment data of customers' first loans.
2. The more frequent the customer uses the loans or the more number of loans the customer starts, the more valuable the customer is. 
3. A customer that defaults within the first several (no more than 3) loans are considered as a not valuable customer.
4. We cannot define whether a customer is valuable if they either just joined the app or only made have very few loans (no more than 3 loans) in his history. 
5. Customers that made a lot of historical loans or made loans frequently in a period/periods are considered as valuable customers.
    1. If a customer churns with default after several loan rounds, he/she is still a valuable customer, but the problem would be that we (the company) fail to offer them customized products. 
    2. If a customer churned (no matter default or not) after a very frequently loan-making period (but in total maybe only 4 or 5 loans), we still consider this is a valuable customer because he/she demonstrated the need, but it is us that fail to offer good customized loan products to retain him/her. 

__Note__: These value propositions (assumptions) are the key ideas in this analysis. While some might be subjective and arbitrary, given the limited data volume, these might be the best we can use to generalize insights from the dataset. __This analysis serves as a great proof-of-concept (POC) and can provide guidance for further optimization when having more dataset.__

## Part 1: Data Cleaning & Exploration

#### 1.1 Configurate the environment & load datasets

In [1]:
# load libraries
import os
import math
import pandas as pd
import numpy as np
import datetime
import statistics
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings

/Applications/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# change work directory 
os.chdir('/Users/craigdavid/Downloads/Facebook Marketing Project/Facebook Marketing Project/Data')
# clear warnings
warnings.filterwarnings('ignore')

In [3]:
# load datasets
dat_ppl = pd.read_csv('people.csv')
dat_repay = pd.read_csv('repayment.csv')
dat_dev = pd.read_csv('device.csv')    
dat_mpesa = pd.read_csv('mpesa.csv')

#### 1.2 Define some functions

Define some frequently used functions.  
Please note: most of the functions in this report requires other libraries to be preloaded. 

In [4]:
# describe the dataset: count, missing, unique_groups
def file_describe(data):
    """
    type: pd.DataFrame
    rtype: pd.DataFrame
    """
    data_describe = pd.DataFrame(data.columns, columns=['colname'])
    data_describe['count'] = data_describe['colname'].apply(lambda x: data[x].count())
    data_describe['missing'] = data_describe['colname'].apply(lambda x: data[x].isnull().sum())
    data_describe['unique_groups'] = data_describe['colname'].apply(lambda x: data[x].unique().size) # include NA
    data_describe.set_index('colname', inplace = True)
    return(data_describe)

In [5]:
# impute missing values with median
def impute_with_median(data):
    """
    type data: Series 
    rtype: Series
    """
    data[data.isnull()] = statistics.median(data[data.notnull()])
    return(data)

In [6]:
# standardize data:
def standardize(data):
    """
    type data: Series
    rtype: Series
    """
    try: 
        if ((data.isnull().sum()==0) & (np.std(data)>0)):
            mu = np.mean(data)
            sd = np.std(data) # assume it is not constant
            return((data-mu)/sd)
        else: 
            raise ValueError('Input data has NA or is constant')
    except:
        raise ValueError('Input datatype is not as specified or input data is constant')

#### 1.3 Clean dataset - dat_repay  
###### Summarize the process:
* Calculate __current loan threshold__:  loan started before (not including) __2015-07-29__ should have been paid back;
* Calculate __nRepayed__: number of paid loans through loanPaidDates;
* Calculate __nRepayRequired__: number of required loan repayments based on nLoans, start date of last loan and current loan threshold 2015-07-29;
* Define different __loan default types__: 
    * __anyDefault__: if the borrower ultimately defaults;
    * __immediateDefault__: if the borrower defaults in the first loan;
    * __noFirstLoanDefault__: if the borrower repays in the first loan;
    * __within3loansDefault__: if the borrower defaults within the first 3 loans;
    * __after7loansDefault__: if the borrower defaults after 7 loans;
    * more types ...
    
__Note__: We will be using __noFirstLoanDefault__ as the target to analyze the likelihood of repayment. As being the target, we will use __immediateDefault__ (the opposite of __noFirstLoanDefault__) to deal with missing values and group very minor buckets in other datasets' mainpulation steps.

In [7]:
# first, take a look at the data quality 
dat_repay_describe = file_describe(dat_repay) # using self-defined file_describe function
dat_repay_describe
# dat_repay.describe(include='all') # pandas built-in describe()

,count,missing,unique_groups
colname,,,
person_id_random,1000,0,1000
nLoans,1000,0,24
loanStartDates,1000,0,871
loanPaidDates,727,273,631
currentDate,1000,0,1


In [8]:
# set person_id_random as the index
dat_repay.set_index('person_id_random', inplace = True)

In [9]:
# var 1: currentDate
currentDate = '2015-08-19'
# the fintech company allows at most 21 days to pay back
currentWindowStart = pd.to_datetime(currentDate) - datetime.timedelta(days = 21) #Timestamp('2015-07-29 00:00:00')
# loan started before (not including) currentWindowStart ('2015-07-29') should have been paid back, vice versa

In [10]:
# var 2: loanPaidDates
# var nRepayed: # of loans have been paid
dat_repay['nRepayed'] = dat_repay['loanPaidDates'].apply(lambda x: len(x.split(';')) if pd.notnull(x) else 0)

In [11]:
# var 3: loanStartDates
# func loan_expire: if a loan should be paid based on the currentDate, return 1, else 0
loan_expire = (lambda x: 1 if pd.to_datetime(x) < currentWindowStart else 0) 
# var nRepayRequired: # of required payment at the currentDate
dat_repay['nRepayRequired'] = dat_repay['loanStartDates'].apply(lambda x: len(x.split(';'))
                                                                +loan_expire(x.split(';')[-1])-1)
# var anyDefault: if nRepayed < nRepayRequired, return True, else False
# Note: sometimes we didn't record the repayments correctly, for example, # of loanStartDates >= # of loanPaidDates + 2
# Therefore, the condition is modified: (nRepayed < nRepayRequired) & (last loanStartDate < currentWindowStart)
# However, id with anyDefault = False does not mean he/she 
# will not commit default in his/her current loan round (if there is a current round), we has to acccept the error here.
dat_repay['anyDefault'] = dat_repay.apply(lambda row: ((row['nRepayRequired']>row['nRepayed']) 
                                                       & (loan_expire(row['loanStartDates'].split(';')[-1]))), axis = 1)
# take a look at the anyDefault:
dat_repay.groupby(['anyDefault'])['anyDefault'].count() # default: 560, paid: 440
# var immediateDefault: default in the first loan, 
dat_repay['immediateDefault'] = dat_repay.apply(lambda row: (row['anyDefault'] 
                                                             & (row['nLoans']==1)), axis=1) #272/560 = 48.6%
# var noFirstLoanDefault: no default in the first loan,
dat_repay['noFirstLoanDefault'] = dat_repay['immediateDefault'].apply(lambda x: 1-x)
# take a look at the anyDefault:
dat_repay.groupby(['immediateDefault'])['immediateDefault'].count() # default: 272, paid: 728, base line: 0.272
# var within3loansDefault: default within the first 3 loans
dat_repay['within3loansDefault'] = dat_repay.apply(lambda row: (row['anyDefault'] 
                                                                & (row['nLoans']<=3)), axis=1) #439/560 = 78.4%
# var after7loansDefault: default after 7 loans,  #36/560 = 6.4%
dat_repay['after7loansDefault'] = dat_repay.apply(lambda row: (row['anyDefault'] 
                                                               & (row['nLoans']>=7)), axis=1) #36/560 = 6.4% 

#### 1.4 Clean dataset - dat_ppl

###### Summarize the process:
* Impute missing values:
    * With median of notnull values, given distribution;
    * Into a dominant groups; 
    * Into higher risk groups/classes; 
    * By creating a new group/class;
* Group minor buckets into one:
    * Similar/close meaning in practice;
    * If they have the similar ratio of the immediateDefault/noFirstLoanDefault (not very minor in this scenario);
    * A few opened-ended answers;

__Note__: Though technically this is not rigorious if we first impute all the data first and then try to build out a statistical model. In a more rigorous setting, we would divide the dataset into training and test set first. However, given the dataset is small, and what we try to learn is what variables are driving a better performance, it is not a must to have a test set to answer these questions. Instead, we use all the records in the model fitting but we make sure the model does not overfit. 

In [12]:
# take a look at the data quality
dat_ppl_describe = file_describe(dat_ppl)
dat_ppl_describe

,count,missing,unique_groups
colname,,,
person_id_random,1000,0,1000
birthday,984,16,940
gender,982,18,3
signup_date,1000,0,1000
fb_friend_count,983,17,710
current_home_years,991,9,7
current_home_months,988,12,13
living_situation,991,9,21
current_job_years,767,233,7


In [13]:
# set person_id_random as index
dat_ppl.set_index('person_id_random', inplace = True)
# merge with dat_repay to facilitate the imputation of NA and grouping buckets
dat_ppl_repay = pd.merge(dat_ppl, dat_repay, left_index=True, right_index=True, how = 'left')

In [14]:
# Check whether high/low missing ratio in dat_ppl is highly related with the target: immediateDefault

# take a look at the records with high missing ratio in dat_ppl
# var numMissing: # of missing fields in the data, attitude-driven records. 
dat_ppl['numMissing'] = dat_ppl.isnull().sum(axis=1)
dat_high_missing_idx = dat_ppl.index[dat_ppl['numMissing']>=8]
dat_ppl_repay.loc[dat_high_missing_idx,'immediateDefault'].mean() # 0.22 in 9 record, note the base line is 0.272

# take a look at the records with no missing values in dat_ppl
dat_no_missing_idx = dat_ppl.index[dat_ppl['numMissing']==0] 
dat_ppl_repay.loc[dat_no_missing_idx,'immediateDefault'].mean() # 0.174 in 466 record

0.17381974248927037

It is quite counterintuitive. We expect the immediateDefault to be a higher number in high missing ratio records. Though the number is such records is very small and might not be significant when we collect more data, we still could have an idea of what is happening here: sometimes people who have the financially ability and willingness to repay don't care much about the survey questions.  

In [15]:
# var 1: birthday
# get approximate age base on current date: 2015-08-19
dat_ppl['age'] = dat_ppl['birthday'].apply(lambda x: 2015 - int(x.split('/')[0]) if pd.notnull(x) else np.nan)
# plt.hist(dat_ppl['age'][dat_ppl['age'].notnull()]), according to figure, impute missing with median
dat_ppl['age'] = impute_with_median(dat_ppl['age'])

In [16]:
# var 2: gender
# impute with the majority group
dat_ppl.loc[dat_ppl['gender'].isnull(), 'gender'] = 'M'

In [17]:
# var 3: fb_friend_count
# impute with the median, according to figure
# plt.hist(dat_ppl['fb_friend_count'][dat_ppl['fb_friend_count'].notnull()]) 
dat_ppl['fb_friend_count'] = impute_with_median(dat_ppl['fb_friend_count']) 

In [18]:
# var 4: education, 26% missing, 6 groups
# impute with a new group: 'unknown'
dat_ppl.loc[dat_ppl['education'].isnull(), 'education'] = 'unknown'
# dat_ppl_repay = pd.merge(dat_ppl, dat_repay, left_index=True, right_index=True, how = 'left')
# dat_ppl_repay.groupby(['education'])['immediateDefault'].agg(['mean', 'count'])
edu_mapping_list = {'masters': 'college or above', # 0.267, 15
                    'college': 'college or above', # 0.177, 515
                    'high_school': 'high_school',  # 0.203, 187
                    'primary': 'low', # 0.091, 22
                    'none': 'none/vacant', # 0, 1
                    'unknown': 'none/vacant' # 0.527, 260
                    } # potentially could be binary: e.g. if provides and has education information
dat_ppl['education'] = dat_ppl['education'].map(edu_mapping_list)

In [19]:
# var 5: current_job_years, current_job_months
dat_ppl.loc[dat_ppl['current_job_years'].isnull(),'current_job_years'] = 'unknown'
dat_ppl.loc[dat_ppl['current_job_years']=='5+','current_job_years'] = '5'
dat_ppl.loc[dat_ppl['current_job_months'].isnull(),'current_job_months'] = 'unknown'
# var current_job_all_blank: if both current_job_years and current_job_months are missing # 231/1000
dat_ppl['current_job_all_blank'] = dat_ppl.apply(lambda row: all(np.in1d(row[['current_job_years', 
                                                                              'current_job_months']], 
                                                                         ['unknown'])), axis = 1)
# if current_job_years is known, current_job_months is unknown, we impute current_job_months with 0
dat_ppl['current_job_months'] = dat_ppl.apply(lambda row: 0 
                                              if (row['current_job_years'] != 'unknown')
                                              &(row['current_job_months']=='unknown') 
                                              else row['current_job_months'], 
                                              axis = 1)
# if current_job_months is known, current_job_years is unknown, we impute current_job_years with 0
dat_ppl['current_job_years'] = dat_ppl.apply(lambda row: 0 
                                             if (row['current_job_months'] != 'unknown')
                                             &(row['current_job_years']=='unknown') 
                                             else row['current_job_years'], 
                                             axis = 1)
# var current_job_years_decimal: aggregate current_job_years and current_job_months, unit: year
dat_ppl['current_job_years_decimal'] = dat_ppl.apply(lambda row: int(row['current_job_years']) 
                                                     + row['current_job_months']/12 
                                                     if not row['current_job_all_blank'] 
                                                     else 'unknown', 
                                                     axis = 1)
# dat_ppl_repay = pd.merge(dat_ppl, dat_repay, left_index=True, right_index=True, how = 'left')
# dat_ppl_repay.loc[dat_ppl_repay['current_job_years_decimal']=='unknown', 'immediateDefault'].mean() 
# 0.186 less than base line: 0.56
# actually ppl without providing current_job_years/months have smaller default rate
# we filling the missing with median, but as 'unknown' has # of 226, this field should not be useful  
dat_ppl['current_job_years_decimal'] = impute_with_median(dat_ppl['current_job_years_decimal'].apply(lambda x: np.nan 
                                                                                                     if x=='unknown' 
                                                                                                     else float(x)))

In [20]:
# var 6: current_home_years, current_home_months
dat_ppl.loc[dat_ppl['current_home_years'].isnull(),'current_home_years'] = 'unknown'
dat_ppl.loc[dat_ppl['current_home_years']=='5+','current_home_years'] = '5'
dat_ppl.loc[dat_ppl['current_home_months'].isnull(),'current_home_months'] = 'unknown'
# var current_home_all_blank: if both current_home_years and current_home_months are missing # 9
dat_ppl['current_home_all_blank'] = dat_ppl.apply(lambda row: all(np.in1d(row[['current_home_years', 
                                                                               'current_home_months']], 
                                                                          ['unknown'])), axis = 1)
# if current_home_years is known, current_home_months is unknown, we impute current_home_months with 0
dat_ppl['current_home_months'] = dat_ppl.apply(lambda row: 0 
                                               if (row['current_home_years'] != 'unknown')
                                               &(row['current_home_months']=='unknown') 
                                               else row['current_home_months'], 
                                               axis = 1)
# if current_home_months is known, current_home_years is unknown, we impute current_home_years with 0
dat_ppl['current_home_years'] = dat_ppl.apply(lambda row: 0 
                                              if (row['current_home_months'] != 'unknown')
                                              &(row['current_home_years']=='unknown') 
                                              else row['current_home_years'], 
                                              axis = 1)
# var current_home_years_decimal: aggregate current_home_years and current_home_months, unit: year
dat_ppl['current_home_years_decimal'] = dat_ppl.apply(lambda row: int(row['current_home_years']) 
                                                      + row['current_home_months']/12 
                                                      if not row['current_home_all_blank'] 
                                                      else 'unknown', 
                                                      axis = 1)
# as dat_ppl['current_home_all_blank'].sum() = 9, we filling the missing with median
dat_ppl['current_home_years_decimal'] = impute_with_median(dat_ppl['current_home_years_decimal'].apply(lambda x: np.nan 
                                                                                                       if x=='unknown' 
                                                                                                       else x))



In [21]:
# var 7: referral_source
# npte: if no referral_source (193), the chance of default is extremely high
# dat_ppl_repay.immediateDefault[dat_ppl_repay['referral_source'].isnull()].mean() # 0.705
dat_ppl_repay.groupby(['referral_source'])['immediateDefault'].agg(['mean', 'count'])
# var if_referred: if referred, return 1, vice versa
dat_ppl['if_referred'] = dat_ppl['referral_source'].apply(lambda x: pd.notnull(x))

In [22]:
# var 8: is_employed
dat_ppl['is_employed'].isnull().sum() # 313
# missing values correspond with significant high default rate 0.476
dat_ppl_repay.immediateDefault[dat_ppl_repay['is_employed'].isnull()].mean() 
# yes 0.180, no 0.175 # and substantially below base line 0.272
dat_ppl_repay.groupby(['is_employed'])['immediateDefault'].agg(['mean', 'count']) 
# impute na with 'unknown'
dat_ppl.loc[dat_ppl['is_employed'].isnull(), 'is_employed'] = 'unknown'
# dat_ppl_repay.loc[dat_ppl_repay['is_employed'].isnull(), 'is_employed'] = 'unknown'
# dat_ppl_repay.groupby(['is_employed'])['immediateDefault'].agg(['mean', 'count']) 

In [23]:
# var 9: relationship_status
dat_ppl['relationship_status'].isnull().sum() # 311
# missing values correspond with significant high default rate 0.479
dat_ppl_repay.immediateDefault[dat_ppl_repay['relationship_status'].isnull()].mean() 
# no much difference, and quite low
dat_ppl_repay.groupby(['relationship_status'])['immediateDefault'].agg(['mean', 'count']) 
# impute na with 'unknown'
dat_ppl.loc[dat_ppl['relationship_status'].isnull(), 'relationship_status'] = 'unknown'
# group married and alternative together as married_alternative: financially good
dat_ppl.loc[dat_ppl['relationship_status'].isin(['alternative', 'married']), 'relationship_status'] = 'married_alternative'
# group single and long-term relationship as single_longterm: financially not good enough for date/marriage
dat_ppl.loc[dat_ppl['relationship_status'].isin(['single', 'long-term']), 'relationship_status'] = 'single_longterm'
# dat_ppl_repay = pd.merge(dat_ppl, dat_repay, left_index=True, right_index=True, how = 'left')
# dat_ppl_repay.groupby(['relationship_status'])['immediateDefault'].agg(['mean', 'count']) 

In [24]:
# var 10: living_situation
dat_ppl['living_situation'].isnull().sum() # 9
dat_ppl_repay.immediateDefault[dat_ppl_repay['living_situation'].isnull()].mean() # 0.22 - only 9 samples
dat_ppl_repay.groupby(['living_situation'])['immediateDefault'].agg(['mean', 'count']) # no much difference & quite low
# func living_sit_map: map the living_situation to 4 groups
def living_sit_map(x):
    if pd.isnull(x):
        return('other') # impute missing value into the highest default rate group
    elif x in ['pay_rent']:
        return('pay_rent')
    elif x in ['own_home']:
        return('own_home')
    elif x in ['with_family', 'student_housing', 'provided for', 
               'Live with my parent', 'I live in my parental home']:
        return('family&stdhouse')
    else:
        return('other')
dat_ppl['living_situation'] = dat_ppl['living_situation'].apply(lambda x: living_sit_map(x))
# dat_ppl_repay = pd.merge(dat_ppl, dat_repay, left_index=True, right_index=True, how = 'left')
# dat_ppl_repay.groupby(['living_situation'])['immediateDefault'].agg(['mean', 'count']) # no much difference, and quite low

In [25]:
# var 11: mpesa_how_often
dat_ppl['mpesa_how_often'].isnull().sum() # 9
# impute na with the majority group
dat_ppl.loc[dat_ppl['mpesa_how_often'].isnull(),'mpesa_how_often'] = 'few_times_a_week' 
# combine the high-frequent group into one: '10+', '4-9', 'daily', 
dat_ppl.loc[dat_ppl['mpesa_how_often'].isin(['10+', '4-9']), 'mpesa_how_often'] = 'daily'
# dat_ppl_repay = pd.merge(dat_ppl, dat_repay, left_index=True, right_index=True, how = 'left')
# dat_ppl_repay.groupby(['mpesa_how_often'])['immediateDefault'].agg(['mean', 'count']) 
# almost no difference between groups

In [26]:
# var 12: outstanding_loan
dat_ppl['outstanding_loan'].isnull().sum() # 5
dat_ppl_repay.immediateDefault[dat_ppl_repay['outstanding_loan'].isnull()].mean() # 0.4 in 5, not a big concern given sample size
dat_ppl.outstanding_loan.describe()
dat_ppl_repay.groupby(['outstanding_loan'])['immediateDefault'].agg(['mean', 'count']) # no: 0.31, yes = 0.19
# impute na with higher risk group
dat_ppl.loc[dat_ppl['outstanding_loan'].isnull(),'outstanding_loan'] = 'no' # assign to group with higher risk 
# dat_ppl_repay = pd.merge(dat_ppl, dat_repay, left_index=True, right_index=True, how = 'left')
# dat_ppl_repay.groupby(['outstanding_loan'])['immediateDefault'].agg(['mean', 'count']) # no: 0.31, yes = 0.19

In [27]:
# var 13: how_much_paid
dat_ppl['how_much_paid'].isnull().sum() # 13
dat_ppl_repay.immediateDefault[dat_ppl_repay['how_much_paid'].isnull()].mean() # 0.23 in 13
dat_ppl.how_much_paid.describe()
# impute na with median
dat_ppl['how_much_paid'] = impute_with_median(dat_ppl['how_much_paid'])

In [28]:
# var 14: how_often_paid
dat_ppl['how_often_paid'].isnull().sum() # 14
dat_ppl_repay.immediateDefault[dat_ppl_repay['how_often_paid'].isnull()].mean() # 0.285 in 14, not a big concern
dat_ppl_repay.groupby(['how_often_paid'])['immediateDefault'].agg(['mean', 'count'])
# impute na with the majority group
dat_ppl.loc[dat_ppl['how_often_paid'].isnull(),'how_often_paid'] = 'monthly'  
# combine the minority group into 'daily' (higher risk)
dat_ppl.loc[~dat_ppl['how_often_paid'].isin(['daily', 'weekly', 'monthly']),'how_often_paid'] = 'daily' 
# dat_ppl_repay = pd.merge(dat_ppl, dat_repay, left_index=True, right_index=True, how = 'left')
# dat_ppl_repay.groupby(['how_often_paid'])['immediateDefault'].agg(['mean', 'count']) 
# monthly has slightly small immediateDefault rate

#### 1.5 Clean dataset - dat_dev
###### Summarize the process:
* Impute missing values or delete missing records based on rules:
* Group minor buckets into one:
* Leave device_model aside at this stage due to its complexity
    * it will be very helpful to just have the price of each device_model since this is a good indicator of the borrower's financial wellness. 
* Set up loan application rules: 
    * __reject loan applications if we don't have enough or any device information__, as missing/no device information is highly correlated with 1st-loan-default. By simplicity, we should have some simple (but robust) rules to reduce the risk substantially.
    * these rules could be improved a lot given more dataset.

__Note__: However, in the survey data, aka, dat_ppl, we cannot do this if people provide less info, because some people, who has the will and ability to pay for the money, might not take the survey for serious. It's true that they need the money, but they are not begging for it to conduct default. And if this is true, they are potentially very valuable customers who frequently take loans.

In [29]:
# take a brief look at the data quality
dat_dev_describe = file_describe(dat_dev)
dat_dev_describe

,count,missing,unique_groups
colname,,,
person_id_random,973,0,973
device_model,972,1,254
hasDualSim,972,1,3
hasWlan,973,0,2
hasBluetooth,973,0,2
mkopo_version_name,972,1,12
android_version,972,1,22


In [30]:
# set the person_id_random as index
dat_dev.set_index('person_id_random', inplace = True)

In [31]:
# Since 27 records are missing in the device dataset
# We assume these are caused by failing to get the device data from the users' sides
# So intuitively these users are more suspicious in behavior, not sharing the device information;

# analyze missing person_id_random, 27 records
no_device_id = set(dat_repay.index) - set(dat_dev.index)
no_device_hist = dat_repay.loc[no_device_id,:]
no_device_hist['immediateDefault'].mean() # 0.778, if no device info, high default rate
# we should have a rule: if no enough device info, reject the loan application (*)

0.7777777777777778

__Insights__: Since missing device information will drive a 1st-loan-default ratio of 78%, we should set a rule: __*reject loan applications if we don't have enough or any device information * __

In [32]:
# var 1: device_model
dat_dev.loc[dat_dev['device_model'].isnull(),:] 
# drop this record as well, considering the rule of enough device information required
drop_index = list(dat_dev.index[dat_dev['device_model'].isnull()])
dat_dev.drop(drop_index, inplace = True)

In [33]:
# var 2: hasDualSim
dat_dev['hasDualSim'].isnull().sum() # 0
# dat_dev.loc[dat_dev['device_model'].isnull(),'hasDualSim'] = 0 # missing rule for future records

0

In [34]:
# var 3: mkopo_version_name
dat_dev['mkopo_version_name'] = dat_dev['mkopo_version_name'].astype(str)
# dat_dev_repay = pd.merge(dat_dev, dat_repay, left_index=True, right_index=True, how = 'left')
# dat_dev_repay.groupby(['mkopo_version_name'])['immediateDefault'].agg(['mean', 'count']) # more latest version -> less default rate
# func mkopo_map: map mkopo_version_name to 3 groups 
def mkopo_map(version):
    if version<='3.0':
        return('old')
    elif version<='3.3':
        return('medium')
    else:
        return('advanced')
# map the mkopo_version_name
dat_dev['mkopoVer'] = dat_dev['mkopo_version_name'].apply(lambda x: mkopo_map(x))
# dat_dev_repay = pd.merge(dat_dev, dat_repay, left_index=True, right_index=True, how = 'left')
# dat_dev_repay.groupby(['mkopoVer'])['immediateDefault'].agg(['mean', 'count']) 

In [35]:
# var 4: android_version
# dat_dev_repay = pd.merge(dat_dev, dat_repay, left_index=True, right_index=True, how = 'left')
# dat_dev_repay.groupby(['android_version'])['immediateDefault'].agg(['mean', 'count']) # more advanced version -> less default rate
# func android_map: map android_version to 3 groups
def android_map(version):
    if version[0:3]<='4.0':
        return('old')
    elif version[0:3]<='4.4':
        return('medium')
    else:
        return('advanced')
# map the android version
dat_dev['andrVer'] = dat_dev['android_version'].apply(lambda x: android_map(x))
# dat_dev_repay = pd.merge(dat_dev, dat_repay, left_index=True, right_index=True, how = 'left')
# dat_dev_repay.groupby(['andrVer'])['immediateDefault'].agg(['mean', 'count']) 

#### 1.6 Clean dataset - dat_mpesa
###### Summarize the process:
* Transform the information from transaction level to customer level;
    * person_id_random
    * count: total transaction number (at most 30)
    * num_out: total transaction out number 
    * num_in: total transaction in number 
    * net_count: (num_out-num_in)/count 
    * avg_amt_out: avg out amount
    * avg_amt_in: avg in amount
    * net_amt_bin: if avg_amt_out>avg_amt_in

__Note__: This dataset helps understand and profile the customers, but considering there is a clear rule in sampling the dataset: the maximum number of transaction provided here is limited to 30. So we should be conservative and only work on simple analyses here. 

In [36]:
# Dataset 4: dat_mpesa (hold)
# some people registered but haven't started any transactions/loads
dat_mpesa_describe = file_describe(dat_mpesa)
dat_mpesa_describe
# impute with median as the missing is minor
dat_mpesa['amount'] = impute_with_median(dat_mpesa['amount'])

In [37]:
# overall
dat_mpesa_all = dat_mpesa.groupby(['person_id_random'])['person_id_random'].agg(['count'])

In [38]:
# subset - out
dat_mpesa_out = dat_mpesa.loc[dat_mpesa.direction == 'out', :]
dat_mpesa_out = dat_mpesa_out.groupby(['person_id_random'])['amount'].agg(['count', 'mean'])
dat_mpesa_out.columns = ['num_out', 'avg_amt_out']

In [39]:
# subset - in
dat_mpesa_in = dat_mpesa.loc[dat_mpesa.direction == 'in', :]
dat_mpesa_in = dat_mpesa_in.groupby(['person_id_random'])['amount'].agg(['count', 'mean'])
dat_mpesa_in.columns = ['num_in', 'avg_amt_in']

In [40]:
# merge together into a person_id level
dat_mpesa_id = pd.merge(dat_mpesa_all, dat_mpesa_out, left_index = True, right_index = True, how = 'left')
dat_mpesa_id = pd.merge(dat_mpesa_id, dat_mpesa_in, left_index = True, right_index = True, how = 'left')

In [41]:
# filter records with na
# since there is a limit of data collect, assuming NAs are caused by random
dat_mpesa_id.drop(dat_mpesa_id.index[dat_mpesa_id.isnull().sum(axis=1)>=1], inplace = True) # 845 id left

In [42]:
# net columns
dat_mpesa_id['net_count'] = (dat_mpesa_id['num_out'] > dat_mpesa_id['num_in'])/dat_mpesa_id['count']
dat_mpesa_id['net_amt_bin'] = dat_mpesa_id['avg_amt_out'] > dat_mpesa_id['avg_amt_in']
# all net_count >=0, 395/845 are equal to 0
# 347 net_amt_bin = True, 498 net_amt_bin is False

<br>

## Part 2: Modeling the Likelihood of Repayment

As we discuss above, the __noFirstLoanDefault__ will be the key indicator in analyzing the __likelihood of repayment__. Recall the __value proposition__ that __if a customer repays his/her 1st loan, it means he/she either is a good customer or has the need to make loans in the future__. On the contrary, if a customer does not repay the 1st loan, it is very likely that they will not repay if offered another loan.   

Honestly, __some people that repay the first loan would default in future loans__. According to this dataset, 560 people default eventually while only 272 of them default in the first loans. If the assumption __"for each borrower, his/her loans are independent from one another"__ is correct, then __noFirstLoanDefault__ would be a almost perfect indicator to measure the likelihood of repayment. But the truth is that this assumption is very strong, especially for the lending business. 

We could construct a complexy metrics based on a combination of many rules to analyze the likelihood of repayment. But this __noFirstLoanDefault__ is a simple and clean way to measure the repayment and help with interpretation in nature. So we move forward with it.  

* __Approach: analyze statistics in logistic regression result__  
* __Target: fit a model that predict the noFirstLoanDefault__



__Note__: We do construct a complexity metrics called __life-time-value (LTV)__ in the Part 3 based on a combination of many rules. We once again put it as a __binary variable__ just as __noFirstLoanDefault__ because the data volume is small and the __discrete metrics would be very volatile__. But that is not directly related with __likelihood of repayment__. Because in the part 3, we want to maximize the repayment outcomes and continuing using the __noFirstLoanDefault__ does not tell anything about the lifetime repayment history. We wil be talking about details in the __Part 3__.


#### 2.1 Prepare the combined dataset

In [43]:
# merge dat_ppl, dat_repay, dat_dev
dat = pd.merge(dat_ppl, dat_dev, left_index=True, right_index=True, how = 'left')
dat = pd.merge(dat, dat_repay, left_index=True, right_index=True, how = 'left')

In [44]:
# filter some records - based on rules
# rule 1: filter records if no device info. 27+1 in this case
dat = dat.loc[dat.isnull().sum(axis=1)<5,:].copy()
# rule 2: filter records if numMissing is too high
# we already impute this field, we hold this filtering later!
# dat1 = dat.loc[dat.numMissing<=8,:].copy() # 963 # Optional filteirng
dat.shape

(972, 43)

In [45]:
# save the merged clean file just in case
dat.to_csv('dat_noFirstLoanDefault_' + 
                 datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S") + 
                 ".csv", index = True,
                 encoding = 'utf-8')

In [46]:
# drop unuseful columns
drop_list = ['birthday', 
             'signup_date', 
             'referral_source',
             'current_home_years', 
             'current_home_months',
             'current_job_years', 
             'current_job_months',
             'current_job_all_blank', 
             'current_home_all_blank',
             'nRepayRequired',
             'device_model', 
             'mkopo_version_name', 
             'android_version']
dat.drop(drop_list, axis = 1, inplace = True)

#### 2.2 Modeling the likelihood of repayment

Since __the likelihood of noFirstLoanDefault (Repayment) + the likelihood of immediateDefault = 1__, we have the variable called 'noFirstLoanDefault' will be the target in this logistic regression model.  

Facebook only offers filters for __(1) mobile device type, (2) minimum Android OS, (3) wireless connection type (mobile, wifi), (4) gender, (5) age, (6) education, (7) work, (8) income, and (9) homeownership__. 

So we will be doing the logistic regression purely based on these informations.  

We choose to neglect the variable mobile device type given it complexity. And we believe that (7) and (9) could be provided as granular as 'is_employed' or 'living_situation', instead of current_job_years/months and current_home_years/months. We move forward with this the following variables. 

In [47]:
# the information we include in the predictive model.
var = ['gender', # binary ['M', 'F']
       'living_situation', # 4 levels ['other', 'pay_rent', 'family&stdhouse', 'own_home']
       'education', # 4 levels ['none/vacant', 'college or above', 'high_school', 'low']
       'how_much_paid', # numeric
       'is_employed', # 3 levels ['unknown', 'yes', 'no']
       'age', # numeric
       'hasWlan', # binary 
       'andrVer' # 3 levels ['old', 'medium', 'advanced']
       ]

In [48]:
# dummy the var if applicable 
dat_model = dat[var].copy()
dat_model = pd.get_dummies(dat_model, drop_first=False)
# set the base of each categorical variables by dropping them 
drop_var_list = ['gender_M', 
                 'living_situation_pay_rent',
                 'education_college or above',
                 'andrVer_old',
                 'is_employed_unknown'
                 ]
dat_model.drop(drop_var_list, axis = 1, inplace = True)
dat_model.shape # (972, 14)
# standardize how_much_paid, 
dat_model['how_much_paid'] = standardize(dat_model['how_much_paid'].apply(lambda x: math.log(x+1)))
# plt.hist(dat_model['how_much_paid'], bins =100) # normal distributed
# standardize age
dat_model['age'] = standardize(dat_model['age'])
# append the target column: noFirstLoanDefault
dat_model['Repayment'] = dat['noFirstLoanDefault']

In [49]:
# check correlation
corr_matrix = dat_model.corr() # correlation checked, decent enough

In [50]:
# append intercept as 1
dat_model['intercept'] = 1   
# run a logistic regression
model = sm.Logit(dat_model['Repayment'], dat_model.loc[:, dat_model.columns != 'Repayment'])
result = model.fit(maxiter=100)
result.summary2()

Optimization terminated successfully.
         Current function value: 0.512592
         Iterations 7


<class 'statsmodels.iolib.summary2.Summary'>
"""
                                 Results: Logit
================================================================================
Model:                    Logit                 No. Iterations:        7.0000   
Dependent Variable:       Repayment             Pseudo R-squared:      0.101    
Date:                     2018-07-16 23:59      AIC:                   1026.4795
No. Observations:         972                   BIC:                   1099.6698
Df Model:                 14                    Log-Likelihood:        -498.24  
Df Residuals:             957                   LL-Null:               -554.15  
Converged:                1.0000                Scale:                 1.0000   
--------------------------------------------------------------------------------
                                  Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
--------------------------------------------------------------------------------
how_much_paid                     0.1902   0.0827  2.3011 0.0214  0.0282  0.3523
age                               0.0503   0.0709  0.7090 0.4783 -0.0887  0.1893
hasWlan                           0.3658   0.1769  2.0677 0.0387  0.0191  0.7125
gender_F                          0.3078   0.1826  1.6853 0.0919 -0.0502  0.6657
living_situation_family&stdhouse  0.1082   0.3274  0.3303 0.7412 -0.5336  0.7499
living_situation_other            0.6901   0.4486  1.5385 0.1239 -0.1891  1.5693
living_situation_own_home         0.2811   0.2218  1.2674 0.2050 -0.1536  0.7158
education_high_school            -0.0476   0.2215 -0.2151 0.8297 -0.4818  0.3865
education_low                     0.8671   0.7554  1.1479 0.2510 -0.6134  2.3476
education_none/vacant            -1.0048   0.3673 -2.7355 0.0062 -1.7247 -0.2849
is_employed_no                    0.4627   0.3794  1.2196 0.2226 -0.2809  1.2062
is_employed_yes                   0.2640   0.3524  0.7493 0.4537 -0.4266  0.9547
andrVer_advanced                  1.8973   1.0533  1.8013 0.0717 -0.1671  3.9617
andrVer_medium                    0.3857   0.2055  1.8767 0.0606 -0.0171  0.7885
intercept                         0.4002   0.3990  1.0030 0.3159 -0.3818  1.1822
================================================================================

"""

Considering that **is_employed_unknown** (1-is_employed_no-is_employed_yes) is highly correlated (corr=0.87) with **education_none/vacant**, so there could be multicollinearity in this problem. we need to iterate to do feature selection and make sure **is_employed_no**, **is_employed_yes** and **education_none/vacant** will not all appear in a same result. 

In [51]:
# Optional feature selection
# we drop variable with the highest p-value each time until every variable is significant
# This further reduce the overfitting and improve generalization power
# run a logistic regression
model = sm.Logit(dat_model['Repayment'], 
                 dat_model.loc[:, ~(dat_model.columns.isin(['Repayment', 
                                                            'education_high_school',
                                                            'living_situation_family&stdhouse',
                                                            'is_employed_yes',
                                                            'age',
                                                            'is_employed_no',
                                                            'education_low',
                                                            'living_situation_own_home']))])
result = model.fit(maxiter=100)
result.summary2() # the result meets expectations
# we don't exclude living_situation_other as it explains a potential direction of user segments.

Optimization terminated successfully.
         Current function value: 0.515648
         Iterations 7


<class 'statsmodels.iolib.summary2.Summary'>
"""
                            Results: Logit
======================================================================
Model:                Logit              No. Iterations:     7.0000   
Dependent Variable:   Repayment          Pseudo R-squared:   0.096    
Date:                 2018-07-16 23:59   AIC:                1018.4205
No. Observations:     972                BIC:                1057.4553
Df Model:             7                  Log-Likelihood:     -501.21  
Df Residuals:         964                LL-Null:            -554.15  
Converged:            1.0000             Scale:              1.0000   
----------------------------------------------------------------------
                        Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
----------------------------------------------------------------------
how_much_paid           0.1771   0.0751  2.3577 0.0184  0.0299  0.3243
hasWlan                 0.3680   0.1759  2.0916 0.0365  0.0232  0.7128
gender_F                0.3001   0.1802  1.6652 0.0959 -0.0531  0.6533
living_situation_other  0.6358   0.4475  1.4207 0.1554 -0.2413  1.5129
education_none/vacant  -1.3227   0.1833 -7.2154 0.0000 -1.6820 -0.9634
andrVer_advanced        1.8499   1.0514  1.7595 0.0785 -0.2108  3.9106
andrVer_medium          0.3856   0.2038  1.8920 0.0585 -0.0138  0.7851
intercept               0.7657   0.2362  3.2411 0.0012  0.3027  1.2287
======================================================================

"""

* __Interpret this summary:__  
    * __how much paid__, has a P-value of 0.0124 and a positive coef. It means the higher the customer gets paid, the more likely he/she will repay.   
    * __gender female__, has a P-value of 0.09 and a postive coef. It means the female customers are more likely to repay than male customers. Though the p-value is not ideally small to conclude with more confidence, it still provides a good direction that it is suggested to target female audience rather than male audience.   
    * __education none or vacant__, has a p-value of 0.0062 and a large negative coef. It means people don't fill this field in the application are significantly less likely to repay. 
    * As __is employed unknown__ is highly correlated with __education none or vacant__, this also suggest that people don't fill is_employed in the application are significantly less likely to repay.  
    * __andriod version__, both 'advanced' and 'medium' have small p-values and positive coefs. We can see that advanced andriod version has a very large coef and while medium andriod version has a relative smaller one. Since the base is the old andriod version, this simply means the higher andriod version, the more likely of repayment.  
    * __hasWlan__, has a positive coef and a very small p-value. It means hasWlan = 1 is more likely to drive higher likelihood of repayment.    
    * __age__, and __living_situation__ do not show a statistically significant impact on the repayment likelihood.  
    
In the real world, __making decisions through blur observations__ is very important. Though not very significant statistically, for exmaple, p-value slightly larger than 0.05, it is still useful to have a sense of how to make the decision if there has to be one.   
    
As the dataset contains 972 records, which is a relatively high record-variable ratio for logisic regression. Overfitting is not a major concern here. So we don't partition the data into training and test. Also, we care more about the coef of each variable rather than whether the model achieves its best predictive power. Simply fitting all the dataset with logistic regression would be enough to generalize the direction of the variables. 

#### 2.3 Support the decisions

In [52]:
dat.groupby(['gender'])['noFirstLoanDefault'].agg(['mean', 'sum', 'count'])

,mean,sum,count
gender,,,
F,0.771536,206,267
M,0.731915,516,705


In [53]:
dat.groupby(['andrVer'])['noFirstLoanDefault'].agg(['mean', 'sum', 'count'])

,mean,sum,count
andrVer,,,
advanced,0.952381,20,21
medium,0.776215,607,782
old,0.562130,95,169


In [54]:
dat.groupby(['hasWlan'])['noFirstLoanDefault'].agg(['mean', 'sum', 'count'])

,mean,sum,count
hasWlan,,,
0.0,0.675325,156,231
1.0,0.763833,566,741


In [55]:
dat.groupby(['is_employed'])['noFirstLoanDefault'].agg(['mean', 'sum', 'count'])

,mean,sum,count
is_employed,,,
no,0.824645,174,211
unknown,0.554386,158,285
yes,0.819328,390,476


In [56]:
dat.groupby(['education'])['noFirstLoanDefault'].agg(['mean', 'sum', 'count'])

,mean,sum,count
education,,,
college or above,0.820755,435,530
high_school,0.796791,149,187
low,0.909091,20,22
none/vacant,0.506438,118,233


In [57]:
# Optional
# we can use a quick random forest to get the variable importance to once again support the logistic regression results

#### 2.4 Understand the customers through dat_mpesa

Integrate the dat with the dat_mpesa_id to understand the behavior

In [58]:
# merge dat and dat_mepsa_id
dat_repay_mpesa_id = pd.merge(dat, dat_mpesa_id, left_index = True, right_index = True, how = 'left')
# consider missing as random, delete them
dat_repay_mpesa_id.drop(dat_repay_mpesa_id.index[dat_repay_mpesa_id.isnull().sum(axis=1)>3], 
                        inplace = True)
dat_repay_mpesa_id.shape # (832, 37)

(832, 37)

In [59]:
# See the transaction number difference by repayment
dat_repay_mpesa_id.groupby(['noFirstLoanDefault'])['count'].agg(['count', 'mean'])

,count,mean
noFirstLoanDefault,,
0,166,18.445783
1,666,25.948949


__Finding 1: Customers that repay first loans on average have more transactions and more active than customers that do not repay first loans__

In [60]:
# See the net_count
dat_repay_mpesa_id.groupby(['noFirstLoanDefault'])['net_count'].agg(['count', 'mean']) 

,count,mean
noFirstLoanDefault,,
0,166,0.039283
1,666,0.025180


__Finding 2: Customers that repay first loans on average have more in-transactions than customers that do not repay first loans__

In [61]:
# See the net_count
dat_repay_mpesa_id.groupby(['noFirstLoanDefault'])['avg_amt_out', 'avg_amt_in'].agg(['mean']) 

,avg_amt_out,avg_amt_in
,mean,mean
noFirstLoanDefault,,
0,1902.099023,2411.226442
1,2522.464387,3477.789643


__Finding 3: Customers that repay first loans on average have larger transaction amount than customers that do not repay first loans__.   

## Part 3: Modeling the Life-Time-Value (LTV) of Borrowers

As we discussed in part 2, analyzing the likelihood of first loan repayment doesn't equal to analyzing the overall repayment outcomes. Paying back the first loan doesn't mean the borrower is a valuable customer.  
<br>
Here, we want to construct a complex metrics called __life-time-value (LTV)__. Usually the __LTV__ should be a discrete number but as the dataset is small, defining a discrete LTV would introduce a lot of variance. So as a workaround, we create a __binary LTV__, just identifying whether the borrower is valuable or not given their past loan history.  
<br>
As we are __creating a new dependent variable (LTV)__, we need to label each record with the corresponding value. Normally this is realized through some experiential __rule-based labeling__ and __manually determining the class of complicated records near the decision boundary__. Here we use the __rule-based labeling__ method to determine the target dependent variable. This approach is bit arbitrary and subjective, but definitely would be useful as a __Proof-of-Concept (POC)__ and improved later for productionalized labeling. 
<br> 

Recall our value proposition in the beginning of the report. We define the profile of three kind of customers based on their __loan frequency__ and __number of historial loans__:

* __High LTV Customers__: meet at least one of the following,
    * frequent loans in a period/ periods based on a significant number of loans;
    * large number of loans in history;
* __Low LTV  Customers__: 
    * Default in first a few loans, for example, within first 3 loans;
* __Unclear LTV Customers__: (to be removed)
    * New customers with no default but just quite a small number of loans, for example, no more than 3 loans;   


We create one more field __LTV__ based on the following rules:
* __Rule 1__: if a customer defaults within the first 3 loans, we label he/she as Low LTV Customer;
* __Rule 2__: if a customer never defaults but has no more than 3 loans, remove the record since LTV is unclear;  We assume these borrowers are randomly distributed and dropping them does not influence the distribution.  
* __Rule 3__: if the median of loan intervals of a customer is within 5 weeks (35 days), label he/she as High LTV Customer ; 
* __Rule 4__: if a customer has over 10 historical loans, label he/she as High LTV Customer, no matter he/she eventually defaults or not. Even if they do default, the total return is almost positive and we should think about ways to retain these customers as they might churn because of not finding a more customized product/loan plan.  

Approach: analyze statistics in logistic regression result   
Target: fit a model that predict the LTV   

__Note__: Those are not the optimal rules, but it labels the customers in a way which is decent enough to empower predictions. But given the small data size and subjective labeling philosophy, we should not try to optimize the prediciton results or interpret too much. Combined with some expertise checking, the rule could be more complex and robust given more dataset.

#### 3.1 Prepare the dataset: dat_ltv

In [62]:
# create a copy to start, as we will drop some records
dat_ltv = dat.copy()
# dat_ltv[['LTV', 'noFirstLoanDefault']].corr() -> corr = 0.62, not very correlated
# Conduct Rule 2:
dat_ltv.drop(dat_ltv.index[((dat_ltv['nLoans']<=3)&(dat_ltv['anyDefault']==0))], inplace = True) # Rule 2
# get the median loan intervals
# assign all within3loansDefault=1 with a extremely high value 999 (unit: months)
dat_ltv['freq'] = 999 
def get_time_intervals(date_string):
    """
    type: string
    rtype: float
    
    example:
    input: date_string = '2018-07-01;2018-07-03;2018-07-05'
    output: 2.0 (unit: day)
    """
    date_list = date_string.split(';')
    if len(date_list) > 1:
        return(statistics.median([(pd.to_datetime(date_list[i+1]) - pd.to_datetime(date_list[i])).days 
                                  for i in range(len(date_list)-1)]))
    else: 
        raise ValueError('Input only contains 1 string')
# calculate the median loan intervals of records with within3loansDefault=0
dat_ltv['freq'] = dat_ltv.apply(lambda row: get_time_intervals(row['loanStartDates']) 
                                if row['within3loansDefault']==0 
                                else row['freq'], 
                                axis = 1)
# Rule 1 & 3:
# 35 days is an arbitrary number in Rule 3
dat_ltv['LTV'] = dat_ltv['freq'].apply(lambda x: 1 if x<35 else 0)
# Rule 4:
dat_ltv.loc[dat_ltv['nLoans']>=10, 'LTV'] = 1 # here actually no change

#### 4.2 Modeling the LTV using logistic regression

In [63]:
# let's do the logistic regression once again
var = ['gender', # binary ['M', 'F']
       'living_situation', # 4 levels ['other', 'pay_rent', 'family&stdhouse', 'own_home']
       'education', # 4 levels ['none/vacant', 'college or above', 'high_school', 'low']
       'how_much_paid', # numeric
       'is_employed', # 3 levels ['unknown', 'yes', 'no']
       'age', # numeric
       'hasWlan', # binary 
       'andrVer' # 3 levels ['old', 'medium', 'advanced']
       ]

# dummy the var if applicable 
dat_ltv_model = dat_ltv[var].copy()
dat_ltv_model = pd.get_dummies(dat_ltv_model, drop_first=False)
# set the base of each categorical variables by dropping them 
drop_var_list = ['gender_M', 
                 'living_situation_pay_rent',
                 'education_college or above',
                 'andrVer_old',
                 'is_employed_unknown'
                 ]
dat_ltv_model.drop(drop_var_list, axis = 1, inplace = True)
dat_ltv_model.shape # (745, 14)
# standardize how_much_paid
dat_ltv_model['how_much_paid'] = standardize(dat_ltv_model['how_much_paid'].apply(lambda x: math.log(x+1)))
# standardize age
dat_ltv_model['age'] = standardize(dat_ltv_model['age'])
# append the target column: LTV
dat_ltv_model['LTV'] = dat_ltv['LTV']


# check correlation
ltv_corr_matrix = dat_ltv_model.corr()  # decent, good enough

# append intercept as 1
dat_ltv_model['intercept'] = 1   
# run a logistic regression
model_ltv = sm.Logit(dat_ltv_model['LTV'], dat_ltv_model.loc[:, dat_ltv_model.columns != 'LTV'])
result_ltv = model_ltv.fit(maxiter=100)
result_ltv.summary2()


Optimization terminated successfully.
         Current function value: 0.647842
         Iterations 5


<class 'statsmodels.iolib.summary2.Summary'>
"""
                                 Results: Logit
================================================================================
Model:                    Logit                 No. Iterations:        5.0000   
Dependent Variable:       LTV                   Pseudo R-squared:      0.054    
Date:                     2018-07-17 00:00      AIC:                   995.2851 
No. Observations:         745                   BIC:                   1064.4859
Df Model:                 14                    Log-Likelihood:        -482.64  
Df Residuals:             730                   LL-Null:               -510.06  
Converged:                1.0000                Scale:                 1.0000   
--------------------------------------------------------------------------------
                                  Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
--------------------------------------------------------------------------------
how_much_paid                     0.2702   0.0870  3.1037 0.0019  0.0996  0.4407
age                               0.0927   0.0778  1.1915 0.2335 -0.0598  0.2452
hasWlan                           0.0340   0.1823  0.1864 0.8521 -0.3233  0.3912
gender_F                          0.3184   0.1776  1.7922 0.0731 -0.0298  0.6665
living_situation_family&stdhouse  0.2785   0.3285  0.8479 0.3965 -0.3653  0.9224
living_situation_other           -0.0106   0.4441 -0.0240 0.9809 -0.8811  0.8598
living_situation_own_home         0.1099   0.2095  0.5245 0.5999 -0.3008  0.5206
education_high_school             0.0050   0.2090  0.0238 0.9810 -0.4046  0.4145
education_low                     0.2237   0.5818  0.3846 0.7005 -0.9165  1.3640
education_none/vacant            -1.0440   0.3401 -3.0698 0.0021 -1.7106 -0.3775
is_employed_no                   -0.5647   0.3386 -1.6676 0.0954 -1.2285  0.0990
is_employed_yes                  -0.4327   0.3119 -1.3874 0.1653 -1.0439  0.1786
andrVer_advanced                  1.1364   0.6637  1.7122 0.0869 -0.1644  2.4372
andrVer_medium                    0.5217   0.2227  2.3423 0.0192  0.0852  0.9583
intercept                        -0.2602   0.3788 -0.6868 0.4922 -1.0026  0.4823
================================================================================

"""

Considering that **is_employed_unknown** (1-is_employed_no-is_employed_yes) is highly correlated (corr=0.85) with **education_none/vacant**, so there could be multicollinearity in this problem. we need to iterate to do feature selection and make sure **is_employed_no**, **is_employed_yes** and **education_none/vacant** will not all appear in a same result. 

In [64]:
# Optional feature selection
# we drop variable with the highest p-value each time until every variable is significant
# This further reduce the overfitting and improve generalization power
# run a logistic regression
model_ltv = sm.Logit(dat_ltv_model['LTV'], 
                 dat_ltv_model.loc[:, ~(dat_ltv_model.columns.isin(['LTV',
                                                                    'education_high_school',
                                                                    'living_situation_other',
                                                                    'hasWlan',
                                                                    #'education_none/vacant',
                                                                    'is_employed_yes',
                                                                    'is_employed_no',
                                                                    'living_situation_family&stdhouse',
                                                                    'living_situation_own_home',
                                                                    'education_low',
                                                                   'age']))])
result_ltv = model_ltv.fit(maxiter=100)
result_ltv.summary2() # the result meets expectations
# we don't exclude living_situation_other as it explains a potential direction of user segments.

Optimization terminated successfully.
         Current function value: 0.651334
         Iterations 5


<class 'statsmodels.iolib.summary2.Summary'>
"""
                           Results: Logit
=====================================================================
Model:                 Logit             No. Iterations:    5.0000   
Dependent Variable:    LTV               Pseudo R-squared:  0.049    
Date:                  2018-07-17 00:00  AIC:               982.4875 
No. Observations:      745               BIC:               1010.1678
Df Model:              5                 Log-Likelihood:    -485.24  
Df Residuals:          739               LL-Null:           -510.06  
Converged:             1.0000            Scale:             1.0000   
---------------------------------------------------------------------
                       Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
---------------------------------------------------------------------
how_much_paid          0.2793   0.0815  3.4257 0.0006  0.1195  0.4391
gender_F               0.3028   0.1739  1.7416 0.0816 -0.0380  0.6436
education_none/vacant -0.6414   0.1902 -3.3726 0.0007 -1.0142 -0.2687
andrVer_advanced       1.1954   0.6560  1.8224 0.0684 -0.0902  2.4810
andrVer_medium         0.5124   0.2202  2.3272 0.0200  0.0809  0.9440
intercept             -0.6035   0.2229 -2.7079 0.0068 -1.0403 -0.1667
=====================================================================

"""

* __Interpret this summary:__  
    * __how much paid__, has a P-value of 0.0006 and a positive coef. It means the higher the customer gets paid, the more likely he/she is a High LTV Customer.   
    * __gender female__, has a P-value of 0.08 and a postive coef. It means the female customers are more likely to be a High LTV Customer. Though the p-value is not ideally small to conclude with more confidence, it still provides a good direction that it is suggested to target female audience rather than male audience.   
    * __education none or vacant__, has a p-value of 0.0007 and a large negative coef. It means people don't fill this field in the application are significantly less likely to be a High LTV Customer. 
    * As __is employed unknown__ is highly correlated with __education none or vacant__, this also suggest that people don't fill is_employed in the application are significantly less likely to be a High LTV Customer. 
    * __andriod version__, both 'advanced' and 'medium' have a relatively small p-values and positive coefs. We can see that advanced andriod version has a large coef and while medium andriod version has a relative smaller one. Since the base is the old andriod version, this simply means the higher andriod version, the more likely of being High LTV Customers.      
    * __age__, __hasWlan__, and __living_situation__ do not show a statistically significant impact on the likelihood of being a High LTV Customer.  
    
Repeat what has been discussed in part 2:   

In the real world, __making decisions through blur observations__ is very important. Though not very significant statistically, for exmaple, p-value slightly larger than 0.05, it is still useful to have a sense of how to make the decision if there has to be one.   
    
As the dataset contains 754 records, which is a relatively high record-variable ratio for logisic regression. Overfitting is not a major concern here. So we don't partition the data into training and test. Also, we care more about the coef of each variable rather than whether the model achieves its best predictive power. Simply fitting all the dataset with logistic regression would be enough to generalize the direction of the variables. 

#### 3.3 Support the decisions

In [65]:
dat_ltv.groupby(['gender'])['LTV'].agg(['mean', 'sum', 'count'])

,mean,sum,count
gender,,,
F,0.472081,93,197
M,0.421533,231,548


In [66]:
dat_ltv.groupby(['education'])['LTV'].agg(['mean', 'sum', 'count'])

,mean,sum,count
education,,,
college or above,0.505348,189,374
high_school,0.467626,65,139
low,0.538462,7,13
none/vacant,0.287671,63,219


In [67]:
dat_ltv.groupby(['is_employed'])['LTV'].agg(['mean', 'sum', 'count'])

,mean,sum,count
is_employed,,,
no,0.434483,63,145
unknown,0.345725,93,269
yes,0.507553,168,331


In [68]:
dat_ltv.groupby(['andrVer'])['LTV'].agg(['mean', 'sum', 'count'])

,mean,sum,count
andrVer,,,
advanced,0.666667,8,12
medium,0.468484,275,587
old,0.280822,41,146


#### 2.4 Understand the customers through dat_mpesa

Integrate the dat with the dat_mpesa_id to understand the behavior

In [69]:
# merge dat_ltv and dat_mepsa_id
dat_ltv_mpesa_id = pd.merge(dat_ltv, dat_mpesa_id, left_index = True, right_index = True, how = 'left')
# consider missing as random, delete them
dat_ltv_mpesa_id.drop(dat_ltv_mpesa_id.index[dat_ltv_mpesa_id.isnull().sum(axis=1)>3], 
                        inplace = True)
dat_ltv_mpesa_id.shape # (631, 39)

(631, 39)

In [76]:
# See the transaction number difference by repayment
dat_ltv_mpesa_id.groupby(['LTV'])['count'].agg(['count', 'mean'])

,count,mean
LTV,,
0,321,21.383178
1,310,27.816129


__Finding 1: Customers that have high LTV on average have more transactions and more active than customers that do not have high LTV__

In [77]:
# See the net_count
dat_ltv_mpesa_id.groupby(['LTV'])['net_count'].agg(['count', 'mean']) 

,count,mean
LTV,,
0,321,0.032441
1,310,0.023643


__Finding 2: Customers that have high LTV on average have more in-transactions than customers that do not have high LTV__

In [78]:
# See the net_count
dat_ltv_mpesa_id.groupby(['LTV'])['avg_amt_out', 'avg_amt_in'].agg(['mean']) 

,avg_amt_out,avg_amt_in
,mean,mean
LTV,,
0,2068.729972,2638.526731
1,2747.704328,3973.168816


__Finding 3: Customers that have high LTV on average have larger transaction amount than customers that do not have high LTV__.  

## 4. Summary of Recommendations

#### 4.1 Recommendation - User Acquisition
* According to model results in analyzing LTV and Repayment, marketing and user acquisition on Facebook should focus on: <br>
    * Devices with higher android version; <br>
    * Female customers; <br>
    * Customers with higher incomes; <br>
    * Customers with clear employment status and education history; <br>
<br>
* Build referral program: <br>
    * If a new customer was referred by any source, he/she is very likely (83%) to repay the first loan; <br>
<br>
* Target customers with more frequent financial activities: <br>
    * Using Mpesa data, we identify that on average high LTV customers transact more frequently; <br>
<br>
* Target customers with stable relationship status: <br>
    * Married people have better repayment as they are more likely to have stable financial condition; <br>

#### 4.2 Recommendation - Loan Application & Product 
* Set up some simple rules during the loan application process; <br>
    * For example, the credit system should reject the loan application if it fails to collect enough device information.  <br>
<br>
* Build out more flexible and customized loan plans (products) for customers; <br>
    * Customers with a few historical loans demonstrate their need of loans; <br>
    * They churn (with a default or not), potentially looking for other peer loan products on the market; <br>
    * Potentially useful products to help with user retention include (not limited to): <br>
        * Set up a credit line (with a slightly higher interest rate), instead of repaying all the amount before making a new loan; it helps retain high LTV customers as they transact more frequently; <br>
        * Add more plans in addition to ‘three weekly repayments’; for example, one monthly repayment would be more appealing as there might be a seasonality effect that customers need loans to pay for monthly bills like house-rent according to ‘signup_date’ field in Data Quality Report (DQR); <br>


### Thanks for reading
### Please reach out to me via cheng.chen.2017@marshall.usc.edu for more details or discussion.